In [54]:
import pandas as ps
import numpy as np
import os 


from trading_robot.connectors.connect_mt5 import ConnectMT5
from trading_robot.data_collection.data_collector import DataCollector

import mt5linux  as MetaTrader5
from trading_robot.utils.logger import log_message
from trading_robot.data_collection.data_collector import DataCollector


In [21]:
mt5 = MetaTrader5.MetaTrader5(host='localhost', port=8001)

account = os.getenv('MT5_ACCOUNT')
password = os.getenv('MT5_PASSWORD')
server = os.getenv('MT5_SERVER')


try:
    mt5.login(login=int(account), password=password, server=server)
    account_info = mt5.account_info()
    rates = mt5.copy_rates_from_pos(symbol="EURUSD", timeframe=mt5.TIMEFRAME_D1, 
                                    start_pos=0, count=10)
    print(rates)
    print(account_info)
finally:
    mt5.shutdown()

None
None


In [100]:
class RiskManager(ConnectMT5):
    """
    The RiskManager class manages trading risks based on specified parameters such as 
    benefit ratio, daily risk, and trade risk.

    Attributes:
        balance (float): The balance of the trading account. If not specified, 
                         the current balance from MetaTrader 5 is used.
        benefit_ratio (float): The benefit ratio. Default is 5.
        risk_per_day (float): Daily risk as a percentage of the balance. Default is 10%.
        risk_per_trade (float): Risk per trade as a percentage of daily risk. Default is 2%.
        money_risk_per_day (float): Amount of money at risk per day. 
                                     If not specified, it is calculated from balance and daily risk.
        money_risk_per_trade (float): Amount of money at risk per trade. 
                                       If not specified, it is calculated from daily risk and trade risk.
    """

    def __init__(self, balance=None, benefit_ratio=5, 
                 risk_per_day=10, risk_per_trade=2, 
                 money_risk_per_day=None, money_risk_per_trade=None):
        """
        Initializes the TradingRiskManager object with the given parameters.

        Parameters:
            balance (float, optional): The balance of the trading account. If None, 
                                       the current balance from MetaTrader 5 is used.
            benefit_ratio (float, optional): The benefit ratio. Default is 5.
            risk_per_day (float, optional): Daily risk as a percentage. Default is 10%.
            risk_per_trade (float, optional): Risk per trade as a percentage of daily risk. Default is 2%.
            money_risk_per_day (float, optional): Amount of money at risk per day. 
                                                  If None, it is calculated from balance and daily risk.
            money_risk_per_trade (float, optional): Amount of money at risk per trade. 
                                                    If None, it is calculated from daily risk and trade risk.
        """
        
        super().__init__()
        
        try:
            # Connect to MetaTrader 5 to get current account information
            self._connect_to_mt5()
            
            # If balance is not specified, get it from MetaTrader 5
            self.balance = balance if balance else mt5.account_info().balance
            
        finally:
            # Close the connection to MetaTrader 5
            mt5.shutdown()
            
        # Set the benefit ratio
        self.benefit_ratio = benefit_ratio
        
        # Set the daily risk percentage
        self.risk_per_day = risk_per_day
        
        # Set the trade risk percentage of daily risk
        self.risk_per_trade = risk_per_trade
        
        # Calculate money risk per day if not specified
        self.money_risk_per_day = money_risk_per_day if money_risk_per_day else self.balance * (risk_per_day / 100)
        
        # Calculate money risk per trade if not specified
        self.money_risk_per_trade = money_risk_per_trade if money_risk_per_trade else self.money_risk_per_day * (risk_per_trade / 100)

    def calculate_trade_parameters(self, symbol, signal,
                                   st_loss_points=100, min_stop_los = None ,
                                   verbose = False):
        """
        Calculates trade parameters such as lot size, stop-loss, and take-profit.

        Parameters:
        - symbol (str): The trading symbol.
        - direction (str): The trading direction ("buy" or "sell").
        - percent_risk (float): The percentage of balance at risk in each trade.
        - st_loss_points (int): The number of points for setting stop-loss (default is 100).
        - benefit_ratio (int): The benefit ratio for setting take-profit (default is 5).
        - balance (float): The account balance (default is None, which uses the balance from the trading account).
        - verbose (bool): If True, print detailed information about the trade parameters (default is False).
        
        Returns:
        - lot_size_to_use (float): The calculated lot size to use in the trade.
        - stop_loss_price (float): The calculated stop-loss price.
        - take_profit_price (float): The calculated take-profit price.
        """ 
        
        account_info, symbol_info = self._get_account_and_symbol_info(symbol)

        st_loss_points = self.__stop_loss(st_loss_points, min_stop_los,
                                 symbol_info.trade_tick_size, 
                                 symbol_info.spread )
            
        # Get entry price based on trading direction
        entry_price = symbol_info.ask if signal == "buy" else symbol_info.bid
         # Check for division by zero
        if entry_price == 0:
            raise ValueError("Function calculate_trade_parameters. Cannot calculate risk per trade due to division by zero.")

        lot_size_to_use = self.__calculate_lot_size(symbol= symbol,
                                                  symbol_info=symbol_info, 
                                                  price=entry_price, 
                                                  stop_loss=st_loss_points, 
                                                  signal=signal )
        
        # Calculate stop-loss and take-profit prices
        decimal_length = len(str(symbol_info.trade_tick_size).split('.')[-1])
        
        if signal == "buy":
            stop_loss_price = round(entry_price - st_loss_points * symbol_info.point, decimal_length)
            take_profit_price = round(entry_price + self.benefit_ratio * st_loss_points * symbol_info.point, decimal_length)
            
        elif signal == "sell":
            stop_loss_price = round(entry_price + st_loss_points * symbol_info.point, decimal_length)
            take_profit_price = round(entry_price - self.benefit_ratio * st_loss_points * symbol_info.point, decimal_length)
        
        else :
            lot_size_to_use = 0
            stop_loss_price = 0
            take_profit_price = 0
            
        # Output results
        try:
            self._connect_to_mt5()
            if self.balance <= account_info.balance - self.money_risk_per_day:
                lot_size_to_use = 0.0
        finally:
            mt5.shutdown()

        if verbose:
            log_message(f"Risk per trade: {self.money_risk_per_day * (self.risk_per_trade / 100)} in currency")
            log_message(f"Lot size to use: {lot_size_to_use}")
            log_message(f"Entry price: {entry_price}")
            log_message(f"Stop-loss price: {stop_loss_price}")
            log_message(f"Take-profit price: {take_profit_price}")

        return lot_size_to_use, stop_loss_price, take_profit_price

    def _get_account_and_symbol_info(self, symbol):
        """
        Internal method for getting account and symbol information from MetaTrader 5.

        Parameters:
        symbol (str): Symbol (e.g. 'EURUSD') to get information for.

        Returns:
        tuple: A tuple of two elements:
        - account_info: Current trading account information.
        - symbol_info: Information about the specified symbol.
        """
        try:
            # Connect to MT5
            self._connect_to_mt5()

            # Get account information
            account_info = mt5.account_info()

            # Get symbol information
            symbol_info = mt5.symbol_info(symbol)

        finally:

            # Shutdown MT5
            mt5.shutdown()

        return account_info, symbol_info

    def __stop_loss(self, st_loss_points, min_stop_loss, trade_tick_size, spread):
        """
        Calculate the stop loss value based on the specified stop loss points.
    
        Parameters:
        - st_loss_points (float): The stop loss value in points.
        - min_stop_loss (float): The minimum stop loss value to be applied.
        - trade_tick_size (float): The tick size for the trade.
        - spread (float): The current spread for the trading pair.
    
        Returns:
        - float: The adjusted stop loss value.
    
        This method calculates the stop loss value based on the specified stop loss points. 
        If the stop loss points are provided as a float, it adjusts them based on the trade 
        tick size. If a minimum stop loss value is provided and it is greater than the adjusted 
        stop loss points, the minimum stop loss value is used instead. The resulting stop loss 
        value is then returned.
        """
        
        if isinstance(st_loss_points, float):
            # Adjust stop loss points based on trade tick size
            st_loss_points = st_loss_points / trade_tick_size
            
            # Determine minimum stop loss value
            min_stop_loss = min_stop_loss if min_stop_loss else spread * 3
            
            # If minimum stop loss is greater than adjusted stop loss points, use minimum stop loss
            if min_stop_loss > st_loss_points:
                st_loss_points = min_stop_loss
                    
        return st_loss_points
                
    def __calculate_lot_size(self, symbol, symbol_info, price, stop_loss, signal):
        """
        Calculate the lot size for a trade based on the provided parameters.
    
        Parameters:
        - symbol (str): The trading symbol.
        - symbol_info (object): Information about the trading symbol.
        - price (float): The current price of the trading symbol.
        - stop_loss (float): The stop loss value.
        - signal (str): The trading signal ("buy" or "sell").
    
        Returns:
        - float: The calculated lot size for the trade.
    
        This method calculates the lot size for a trade based on the provided parameters.
        It takes into account various factors such as the risk per trade, stop loss value,
        and currency pair information. The lot size is calculated differently for Forex pairs
        and non-Forex symbols. For Forex pairs, it considers the currency profit currency.
        For non-Forex symbols, it may use an auxiliary currency pair for calculations.
    
        Note:
        This method assumes that the necessary data, such as symbol information and
        auxiliary currency pair data, are available and correctly provided.
    
        """
    
        # Check if the symbol is a Forex pair
        if "forex" in symbol_info.path.lower():
            if "USD" in symbol:
                if "USD" == symbol_info.currency_profit:
                    # Calculate lot size for USD-based Forex pairs
                    lot = (self.money_risk_per_trade / (stop_loss * symbol_info.point)) / symbol_info.trade_contract_size
                elif "USD" != symbol_info.currency_profit:
                    # Calculate lot size for non-USD-based Forex pairs
                    lot = ((self.money_risk_per_trade * price) / (stop_loss * symbol_info.point)) / symbol_info.trade_contract_size
            elif "USD" not in symbol:
                # Find an auxiliary currency pair with USD for calculation
                data_collector = DataCollector()
                auxiliary_course = [symbol for symbol in data_collector.get_all_symbols()
                                    if symbol_info.currency_profit in symbol and "USD" in symbol][0]
                try:
                    # Connect to MT5
                    self._connect_to_mt5()
                    # Get symbol information for the auxiliary currency pair
                    auxiliary_course_symbol_info = mt5.symbol_info(auxiliary_course)
                finally:
                    # Shutdown MT5
                    mt5.shutdown()
    
                entry_price = auxiliary_course_symbol_info.ask if signal == "buy" else auxiliary_course_symbol_info.bid
                if entry_price == 0:
                    raise ValueError("Function calculate_trade_parameters. Cannot calculate risk per trade due to division by zero.")
    
                # Calculate lot size using the auxiliary currency pair
                if "USD" == auxiliary_course_symbol_info.currency_profit:
                    lot = (self.money_risk_per_trade / ((stop_loss * symbol_info.point) * entry_price)) * symbol_info.point
                elif "USD" != auxiliary_course_symbol_info.currency_profit:
                    lot = (self.money_risk_per_trade * entry_price) / (stop_loss * symbol_info.point) * symbol_info.point
    
        else:
            # Calculate lot size for non-Forex symbols
            lot = ((self.money_risk_per_day * (self.risk_per_trade / 100)) / (stop_loss * symbol_info.trade_contract_size))
    
        # Round the lot size and ensure it is not less than the minimum volume
        lot = round(lot, len(str(symbol_info.volume_min)) - 2)
        lot = max(lot, symbol_info.volume_min)
    
        return lot
        
    def __calculate_atr(self, symbol, timeframe=mt5.TIMEFRAME_D1, period=22):
        """
        Calculate the Average True Range (ATR) for the specified symbol and timeframe.
    
        Parameters:
        - symbol (str): The symbol for which to calculate the ATR.
        - timeframe (int): The timeframe for the ATR calculation (default is mt5.TIMEFRAME_D1).
        - period (int): The period over which to calculate the ATR (default is 250).
    
        Returns:
        - float: The last calculated ATR value.
    
        This method calculates the Average True Range (ATR) for the specified symbol and timeframe.
        It retrieves historical data for the symbol, calculates the true range for each period,
        calculates the mean true range over the specified period, and returns the last calculated
        ATR value.
        """
    
        # Retrieve historical data for the symbol and timeframe
        data_collector = DataCollector()
        data = data_collector.get_historical_data(symbol, timeframe)
    
        # Calculate the mean ATR
        data['Mean_ATR'] = abs(data['Close'] - data['Close'].shift(periods=1)) \
            .fillna(0.01) \
            .rolling(window=period, min_periods=1).mean()
    
        # Round the last ATR value to the same precision as the close price
        last_atr = round(data.iloc[-1, -1], len(str(data.iloc[-1, 2]).split(".")[-1]))
    
        return last_atr



In [ ]:
balance = 3000
rm = RiskManager(balance=balance)
symbol = "EURUSD"
signal = "buy"
rm.calculate_trade_parameters(
    symbol=symbol, 
    signal=signal,
    st_loss_points=100, 
    min_stop_los = None ,
    verbose = True
    )

In [101]:
class Trading(RiskManager):
    """
    A class for managing trading operations with MetaTrader 5, including executing trades,
    placing grid orders, and closing positions or orders. Inherits from RiskManager to utilize 
    risk management features.

    Attributes:
    - balance (float): The trading account balance.
    - benefit_ratio (float): The ratio of benefit to risk.
    - risk_per_day (float): The daily risk percentage.
    - risk_per_trade (float): The risk percentage per trade.
    - money_risk_per_day (float, optional): The monetary risk per day.
    - money_risk_per_trade (float, optional): The monetary risk per trade.
    """

    def __init__(self, balance=None, benefit_ratio=5, 
                 risk_per_day=10, risk_per_trade=2, 
                 money_risk_per_day=None, money_risk_per_trade=None):
        """
        Initialize the Trading class.

        Args:
        - balance (float | None): Account balance. If None, default value is used.
        - benefit_ratio (int): Benefit ratio. Default value is 5.
        - risk_per_day (int): Risk per day in percent. Default value is 10.
        - risk_per_trade (int): Risk per trade in percent. Default value is 2.
        - money_risk_per_day (float | None): Financial risk per day. If None, default value is used.
        - money_risk_per_trade (float | None): Financial risk per trade. If None, default value is used.
        """
        
        # Initialize RiskManager with parameters
        super().__init__(balance=balance, 
                         benefit_ratio=benefit_ratio, 
                         risk_per_day=risk_per_day, 
                         risk_per_trade=risk_per_trade, 
                         money_risk_per_day=money_risk_per_day, 
                         money_risk_per_trade=money_risk_per_trade)
        
    def execute_trading(self, symbol: str, signal: str, lot: int | float = None,  
                        sl: float = None, tp: float = None, deviation: int = 10, 
                        close_previous_deal: bool = True, order_type: str = "market", 
                        order_offset: int = 100) -> bool:
        """
        Executes a trading operation for the given symbol based on the provided signal.

        Args:
        - symbol (str): The trading symbol (e.g., 'EURUSD').
        - signal (str): The trading signal ('buy' or 'sell').
        - lot (int | float, optional): The lot size for the trade. Defaults to None.
        - sl (float, optional): Stop loss level in price points. Defaults to None.
        - tp (float, optional): Take profit level in price points. Defaults to None.
        - deviation (int, optional): Allowed price deviation for the order execution. Defaults to 10.
        - close_previous_deal (bool, optional): If True, closes previous positions or orders. Defaults to True.
        - order_type (str, optional): The type of order to execute ('market' or 'limit'). Defaults to 'market'.
        - order_offset (int, optional): Offset in points for the limit order. Defaults to 100.

        Returns:
        - bool: True if the trading operation is successfully executed.

        Example:
        >>> trading = Trading(balance=10000)
        >>> trading.execute_trading(symbol="EURUSD", signal="buy", lot=1.0, sl=20, tp=50)
        Order executed successfully.
        """

        try:
            # Validate the signal
            if signal not in ["buy", "sell"]:
                raise ValueError("Signal not recognized. Must be 'buy' or 'sell'.")

            # Close previous positions or orders if needed
            if close_previous_deal:
                positions, positions_signal = self.__check_trade_conditions(signal=signal, symbol=symbol, check_type="positions")
                orders, orders_signal = self.__check_trade_conditions(signal=signal, symbol=symbol, check_type="orders")

                if positions_signal:
                    self.close(symbol=symbol, positions=positions)
                    
                if orders_signal:
                    self.close(symbol=symbol, orders=orders)

            # Calculate lot size, stop loss, and take profit if not provided
            if lot is None or sl is None or tp is None:
                lot_size, stop_loss_price, take_profit_price = self.calculate_trade_parameters(symbol=symbol, signal=signal)
                lot = lot if lot is not None else lot_size
                sl = sl if sl is not None else stop_loss_price
                tp = tp if tp is not None else take_profit_price

            # Create an order request based on the signal and other parameters
            if order_type == "market":
                request = self.__market_order(
                    signal=signal, 
                    symbol=symbol, 
                    lot=lot, 
                    sl=sl, 
                    tp=tp, 
                    deviation=deviation
                )
            
            elif order_type == "limit":
                request = self.__limit_order(
                    signal=signal, 
                    symbol=symbol, 
                    lot=lot, 
                    sl=sl, 
                    tp=tp, 
                    deviation=deviation, 
                    shifts=order_offset
                )
            else:
                raise ValueError("Invalid order_type. Must be 'market' or 'limit'.")

            # Send the order request
            result = self.__order_send(request)

            return True

        except Exception as e:
            print(f"Error executing trade: {str(e)}")
            return False

    def place_grid_orders(self, symbol: str, signal:str , direction: str, 
                          grid_count:int = 5, grid_size: int|float = 100, 
                          stop_loss: int|float = 100, take_profit:int|float = 500,
                          lot_size: float|None = None, deviation: int = 10) -> None:

        """
        Places a series of grid limit orders based on the provided parameters.

        Args:
        - symbol (str): Trading symbol (e.g., 'EURUSD').
        - signal (str): Trading signal ('buy' or 'sell').
        - direction (str): Grid direction ('by' for in the direction of the signal, 
                            'against' for against the direction of the signal).
        - grid_count (int): Number of grid levels to place. Defaults to 5.
        - grid_size (int | float): Size of the grid step (in points). Defaults to 100.
        - stop_loss (int | float): Stop loss level (in points). Defaults to 100.
        - take_profit (int | float): Take profit level (in points). Defaults to 500.
        - lot_size (float | None): Size of the trading lot. If None, will be calculated. Defaults to None.
        - deviation (int): Maximum deviation allowed when executing a trade. Defaults to 10.

        Returns:
        - None: This method does not return anything but prints the order request for each grid level.

        Example:
        >>> trading = Trading(balance=10000)
        >>> trading.place_grid_orders(symbol="EURUSD", signal="buy", direction="by", grid_count=3, grid_size=50)
        ```
        """

        # Get the current market price based on the signal
        price = self._get_symbol_price(symbol, signal)
        
        # Calculate lot size if not provided
        if lot_size is None:
            lot_size, *_ = self.calculate_trade_parameters(symbol=symbol, signal=signal)
        
        # Retrieve symbol information (e.g., point size)
        _, symbol_info = self._get_account_and_symbol_info(symbol)

        # Convert grid_size, stop_loss, and take_profit to point-based values if they are integers
        if isinstance(grid_size, int):
            grid_size *= symbol_info.point
        
        if isinstance(stop_loss, int):
            stop_loss *= symbol_info.point
        
        if isinstance(take_profit, int):
            take_profit *= symbol_info.point

        # Place limit orders for each grid level
        for grid_level in range(1, grid_count + 1):
            # Calculate the limit price, stop loss, and take profit for the current grid level
            limit_price, stop_loss_price, take_profit_price = self.__calculate_grid_orders(
                signal=signal,
                price=price,
                grid_level=grid_level,
                grid_size=grid_size,
                stop_loss=stop_loss,
                take_profit=take_profit,
                direction=direction
            )
            
            # Create the limit order request
            request = self.__limit_order(
                price=limit_price,
                signal=signal,
                symbol=symbol,
                lot=lot_size,
                sl=stop_loss_price,
                tp=take_profit_price,
                deviation=deviation,
                shifts=0.0
            )

            # Send a trade order request to MetaTrader 5.
            self.__order_send(request)

    def close(self, symbol, positions=None, orders=None):
        """
        Closes positions and orders for the specified symbol.

        Args:
        - symbol (str): Trading symbol.
        - positions (list, optional): List of open positions for the specified symbol. Defaults to None.
        - orders (list, optional): List of pending orders for the specified symbol. Defaults to None.
        
        Returns:
        - bool: True if the trading operation was successful, False otherwise.

        Example:
        >>> trading = Trading(balance=10000)
        >>> trading.close(symbol="EURUSD", positions=[...])
        True
        """

        if positions :
            self.__close_positions(symbol, positions)
        if orders :
            self.__close_orders(symbol, orders )
            
        return True

    def _get_symbol_price(self, symbol: str, signal: str) -> float:
        """
        Returns the current price for a symbol based on a trading signal.

        Args:
        - symbol (str): Trading symbol (e.g., 'EURUSD').
        - signal (str): Trading signal ('buy' or 'sell').

        Returns:
        - float: Price corresponding to the signal ('ask' for 'buy' and 'bid' for 'sell').

        Example:
        >>> trading = Trading(balance=10000)
        >>> price = trading._get_symbol_price(symbol="EURUSD", signal="buy")
        >>> print(price)
        1.1200
        """
        
        _, symbol_info = self._get_account_and_symbol_info(symbol)
        price = symbol_info.ask if signal == "buy" else symbol_info.bid

        return  price

    def __order_send(self, request: dict) -> None:
        """
        Private method to send a trade order request to MetaTrader 5.

        Args:
        - request (dict): The order request dictionary containing trade parameters.

        Returns:
        - None
        """
        try:
            # Connect to MetaTrader 5
            self._connect_to_mt5()

            # Send the order request to execute the trade
            result = mt5.order_send(request)

            # Check if the result of the order execution is not None
            if result is not None:
                # If the order was not executed successfully, handle the error
                if result.retcode != mt5.TRADE_RETCODE_DONE:
                    print(f"order_send failed, retcode={result.retcode}, comment={result.comment}")
                    
                    # Convert the result to a dictionary for better readability
                    result_dict = result._asdict()
                    
                    # Print detailed information about the trade request if there was an error
                    for field in result_dict.keys():
                        if field == "request":
                            traderequest_dict = result_dict[field]._asdict()
                            for tradereq_field in traderequest_dict:
                                print(f"   traderequest: {tradereq_field}={traderequest_dict[tradereq_field]}")
                else:
                    # If the order was executed successfully, print a success message
                    print("Order executed successfully.")
            else:
                # If the result is None, indicate that the order send failed
                print(f"Order send failed, result is None.")
        except Exception as e:
            # Catch and print any exceptions that occur during order sending
            print(f"__order_send error: {e}")
        finally:
            # Ensure that MetaTrader 5 is shut down after the order is processed
            mt5.shutdown()

    def __limit_order(self, signal: str, symbol: str, lot: float, 
                      sl: float, tp: float, deviation: int,
                        shifts:int|float =100, price:float|bool = None) -> dict:
        """
        Private method to create a limit order request.

        Args:
        - signal (str): Trading signal ("buy" or "sell").
        - symbol (str): Trading symbol (e.g., 'EURUSD').
        - lot (float): Trading lot size.
        - sl (float): Stop loss level.
        - tp (float): Take profit level.
        - deviation (int): Maximum deviation allowed when executing a trade.
        - shifts (int | float): Number of points to adjust the price, stop loss, 
        and take profit levels. Defaults to 100.
        - price (float | None): Specific price for the limit order. If None, 
        the price will be determined based on the current market price.

        Returns:
        - dict: A dictionary representing the order request, ready to be sent to the trading platform.
        """

        if price is None:
            price = self._get_symbol_price(symbol, signal)
            
        if isinstance(shifts, int ):
            _, symbol_info = self._get_account_and_symbol_info(symbol)
            shifts = shifts * symbol_info.point

        price = price - shifts if signal == "buy" else price + shifts

        try:
            self._connect_to_mt5()
            trade_type = mt5.ORDER_TYPE_BUY_LIMIT if signal == "buy" else mt5.ORDER_TYPE_SELL_LIMIT
        finally:
            mt5.shutdown
            
        sl = sl - shifts if signal == "buy" else sl + shifts
        tp = tp - shifts if signal == "buy" else tp + shifts

        request = {
            "action": mt5.TRADE_ACTION_PENDING,
            "symbol": symbol,
            "volume": float(lot),
            "type": trade_type,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": deviation,
            "magic": 234000,
            "comment": "Limit Python order",
            "type_time": mt5.ORDER_TIME_GTC,  
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        
        return request

    def __market_order(self, signal: str, symbol: str, lot: float, 
                       sl: float, tp: float, deviation: int) -> dict:
        """
        Private method to create a market order request.

        Args:
        - symbol_info (mt5 symbol info): Information about the trading symbol.
        - signal (str): Trading signal ("buy" or "sell").
        - symbol (str): Trading symbol.
        - lot (float): Trading lot size.
        - sl (float): Stop loss level.
        - tp (float): Take profit level.
        - deviation (int): Maximum deviation allowed when executing a trade.

        Returns:
        - dict: A dictionary representing the order request, ready to be sent to the trading platform.
        """
        
        price = self._get_symbol_price(symbol, signal)

        # Define the order parameters based on the given signal
        try:
            self._connect_to_mt5()
            trade_type = mt5.ORDER_TYPE_BUY if signal == "buy" else mt5.ORDER_TYPE_SELL
        finally:
            mt5.shutdown

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": float(lot), 
            "type": trade_type,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": deviation,
            "magic": 234000,
            "comment": "Market Python order",
            "type_time": mt5.ORDER_TIME_DAY,
            "type_filling": mt5.TRADE_ACTION_DEAL,
            "request_actions": mt5.TRADE_ACTION_DEAL
        }
        
        return request

    def __close_positions(self, symbol: str, positions: list)-> None:
        """
        Private method to close positions for the specified symbol.

        Args:
        - symbol (str): Trading symbol.
        - positions (list): List of open positions for the specified symbol.
        """

        _, symbol_info = self._get_account_and_symbol_info(symbol)
        
        if positions:
            position = positions[0]
        else:
            return None

        if position.type == 0:
            type = mt5.ORDER_TYPE_SELL
            price = symbol_info.bid
        else:
            type = mt5.ORDER_TYPE_BUY
            price = symbol_info.ask
            
            
        close_request = {
            "action": mt5.TRADE_ACTION_DEAL, 
            "position": position.ticket,
            "symbol": symbol,
            "volume": position.volume,
            "deviation": 10,
            "magic": position.magic,
            "price": price,
            "type": type,
            "comment": "python script close",
            "type_filling": mt5.ORDER_FILLING_IOC,
            "type_time": mt5.ORDER_TIME_GTC,
        }

        result = mt5.order_send(close_request)

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order closing failed: {result.retcode}")
        else:
            print("closed correctly")

    def __close_orders(self, symbol: str, orders:list ) -> None:
        """
        Private method to close orders for the specified symbol.

        Args:
        - symbol (str): Trading symbol.
        - orders (list): List of pending orders for the specified symbol.
        """
        
        for order in orders:
            if order.symbol == symbol:
                request = {
                    "action": mt5.TRADE_ACTION_REMOVE,
                    "order": order.ticket,
                    "magic": 234000,
                    "comment": "Delete order"
                }
                
                result = mt5.order_send(request)
            
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"Order {order.ticket} for symbol {symbol} was successfully deleted.")
                else:
                    print(f"Error when deleting order {order.ticket} by symbol {symbol}: {result.comment}")

    def __calculate_grid_orders(self, signal: str, price: float, grid_level: int, 
                                grid_size: float, stop_loss: float, take_profit: float, 
                                direction: str) -> tuple[float, float, float]:
        """
        Calculates the limit price, stop loss price, and take profit price 
        for a grid trading order based on the provided parameters.

        Parameters:
        signal (str): Trading signal ('buy' or 'sell') indicating the trade direction.
        price (float): Current market price of the symbol.
        grid_level (int): The grid level indicating how far the order is from the current price.
        grid_size (float): The size of the grid step (the distance between grid levels).
        stop_loss (float): Stop loss in price points.
        take_profit (float): Take profit in price points.
        direction (str): The direction of the grid order ('by' for in the direction of the signal, 
                        'against' for against the direction of the signal).

        Returns:
        tuple[float, float, float]: A tuple containing:
            - limit_price (float): The calculated price for the limit order.
            - stop_loss_price (float): The calculated price level for the stop loss.
            - take_profit_price (float): The calculated price level for the take profit.
        """

        # Determine limit price based on the signal and direction
        if signal == "buy":
            if direction == "by":
                limit_price = price + (grid_level * grid_size)
            elif direction == "against":
                limit_price = price - (grid_level * grid_size)

            # Calculate stop loss and take profit prices for a buy order
            stop_loss_price = limit_price - stop_loss
            take_profit_price = limit_price + take_profit

        elif signal == "sell":
            if direction == "by":
                limit_price = price - (grid_level * grid_size)
            elif direction == "against":
                limit_price = price + (grid_level * grid_size)

            # Calculate stop loss and take profit prices for a sell order
            stop_loss_price = limit_price + stop_loss
            take_profit_price = limit_price - take_profit

        return limit_price, stop_loss_price, take_profit_price

    def __check_trade_conditions(self, symbol: str, signal: str, check_type: str) -> tuple:
        """
        Checks if the current open positions or pending orders for a given symbol require switching the trading signal.

        Args:
        - symbol (str): The trading symbol (e.g., 'EURUSD').
        - signal (str): The trading signal ('buy' or 'sell').
        - check_type (str): The type of check to perform ('positions' or 'orders').

        Returns:
            - tuple: A tuple containing:
                - list: The list of positions or orders.
                - bool: True if the signal needs to be switched based on the check, False otherwise.
            """
        
        try:
             # Connect to MetaTrader 5
            self._connect_to_mt5()

            # Define the signal types for positions and orders
            type_signal = {
                "positions": {mt5.POSITION_TYPE_BUY: "buy", mt5.POSITION_TYPE_SELL: "sell"},
                "orders": {mt5.ORDER_TYPE_BUY_LIMIT: "buy", mt5.ORDER_TYPE_SELL_LIMIT: "sell"}
            }
            
            # Retrieve positions or orders based on check_type
            if check_type == "positions":
                items = mt5.positions_get(symbol=symbol)
            elif check_type == "orders":
                items = mt5.orders_get(symbol=symbol)
            else:
                raise ValueError("Invalid check_type. Must be 'positions' or 'orders'.")

            # Check if there's a need to switch the signal
            if items:
                return items, type_signal[check_type][items[0].type] != signal
            
            return items, False
        
        except Exception as e:
            print(f"Error in __check_trade_conditions: {str(e)}")
            return None, False

        finally:
            # Ensure MT5 connection is closed
            mt5.shutdown()


In [ ]:
data_col = DataCollector()
symbol = "EURUSD"
data = data_col.get_historical_data(symbol=symbol)

data["Signal"] = "sell"
# data["Signal"] = "buy"
signal = data["Signal"].iloc[-1]

order_type = 'market'
# order_type = 'limit'

traid = Trading()
traid.execute_trading(signal=signal, symbol=symbol, order_type=order_type)

In [ ]:
data_col = DataCollector()
symbol = "EURUSD"
data = data_col.get_historical_data(symbol=symbol)

data["Signal"] = "sell"
# data["Signal"] = "buy"
signal = data["Signal"].iloc[-1]

direction = 'by'
# direction = 'against'

traid = Trading()
traid.place_grid_orders(signal=signal, symbol=symbol, direction=direction)